In [ ]:
#Download the data

import kagglehub
import shutil
import os

# Download to kagglehub's default cache location
cached_path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print('default path: ',cached_path)
# Your desired custom folder
destination = "../data/raw/"

# Copy dataset to your preferred location
# shutil.copytree(cached_path, destination, dirs_exist_ok=True)
shutil.move(cached_path, destination)

print(f"Dataset copied to: {destination}")


default path:  C:\Users\Amarjeet\.cache\kagglehub\datasets\paultimothymooney\chest-xray-pneumonia\versions\2


KeyboardInterrupt: 